# <div align="center">CP322-A Mini-Project 2: Machine Learning</div>
## <div align="center">Group 6</div>
### <div align="center">due on 12-Nov-2023 at 11:30 PM</div>

Imports:

In [1]:
import numpy as np
import os
import gzip

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

## Datasets

### Dataset 1 (20 Newsgroup): 

Use the default train subset (subset=‘train’, and remove=([‘headers’, ‘footers’, ‘quotes’]) in  sklearn.datasets) to train the models and report the final performance on the test subset.  note: you need to start with the text data and convert the text to feature vectors. Please refer to https://scikitlearn.org/stable/tutorial/text_analytics/working_with_text_data.html for a tutorial on the steps needed for this.

In [2]:
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']

twenty_train = fetch_20newsgroups(subset='train',
    categories=categories, shuffle=True, random_state=42)

d1_data = twenty_train.data
d1_labels = twenty_train.target

tfidf_vectorizer = TfidfVectorizer()
d1_tfidf = tfidf_vectorizer.fit_transform(d1_data)

count_vect = CountVectorizer()

d1_train_counts = count_vect.fit_transform(twenty_train.data)
d1_train_counts.shape

d1_tf_transformer = TfidfTransformer(use_idf=False).fit(d1_train_counts)
d1_train_tf = d1_tf_transformer.transform(d1_train_counts)
d1_train_tf.shape

(2257, 35788)

### Dataset 2 (IMDB Reviews):

In [3]:
def read_imdb():
    print("Begin reading files.")
    directory = 'data/aclImdb/train/neg'
    data = []
    labels = []
    file = ''
    for filename in os.listdir(directory):
        f = open(os.path.join(directory, filename), encoding="utf-8")
        file = filename, "neg"
        for i in f:
            test = True
            while test:
                test = False
                k = i.find('<br /><br />')
                if k != -1:
                    i = i[:k] + ' ' + i[k+12:]
                    test = True

            data.append(i)
            labels.append(0)

    print("Done reading negative values.")

    directory = 'data/aclImdb/train/pos'

    for filename in os.listdir(directory):
        f = open(os.path.join(directory, filename), encoding="utf-8")
        file = filename, "pos"
        for i in f:

            test = True
            while test:
                test = False
                k = i.find('<br /><br />')
                if k != -1:
                    i = i[:k] + ' ' + i[k+12:]
                    test = True

            data.append(i)
            labels.append(1)
        
    print("Done reading positive values.")
    npdata = np.array(data)
    nplabels = np.array(labels)
    
    return npdata, nplabels
  
d2_data, d2_labels = read_imdb()
print("File reading complete.")

Begin reading files.
Done reading negative values.
Done reading positive values.
File reading complete.


In [4]:
tfidf_vectorizer = TfidfVectorizer()
count_vect = CountVectorizer()
d2_train_counts = count_vect.fit_transform(d2_data)
d2_train_counts.shape
d2_tfidf = tfidf_vectorizer.fit_transform(d2_data)

d2_tf_transformer = TfidfTransformer(use_idf=False).fit(d2_train_counts)
d2_imdb_train_tf = d2_tf_transformer.transform(d2_train_counts)
d2_imdb_train_tf.shape

(25000, 74849)

## Models

### Logistic Regression

In [28]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import StratifiedKFold

# Fetch the 20 newsgroups dataset
# newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

# Partiall dataset
# categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']


# Fetch the "train" subset of the data to be used
# newsgroups = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))
newsgroups = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

# Vectorizing the text from the given dataset using TF-IDF to get the matrix
data_vectorizer = TfidfVectorizer(max_features=10000)
X_tfidf = data_vectorizer.fit_transform(newsgroups.data)

# Defining the k-fold cross-validation with 5 folds measures to obtain the 5 groups
kFold = StratifiedKFold(n_splits=5,shuffle=True, random_state=42)

# Defining logistic regression model with 1000 interations
logistic_regression = LogisticRegression(max_iter=1000)

# Defining place holder lists for the metrics for each of the resulting folds
accuracies_array = []
precisions_array = []
recalls_array = []
# Implementing k-fold cross-validation using the test and train data
for train_position, test_position in kFold.split(X_tfidf, newsgroups.target):
    X_trainData= X_tfidf[train_position]
    X_testData = X_tfidf[test_position]
    y_trainData = newsgroups.target[train_position]
    y_testData= newsgroups.target[test_position]

    
    #Using the training portion of data to train it for our Logistic regression model
    logistic_regression.fit(X_trainData, y_trainData)

    # Defining our predicts on the test data of the dataset
    y_predictData = logistic_regression.predict(X_testData)

    # Analysing the models using each of the k-folds
    accuracy = accuracy_score(y_testData, y_predictData)
    classification_report_dict = classification_report(y_testData, y_predictData, output_dict=True)
    
    
    # Adding the metrictics int the defined lists to be used for final average
    accuracies_array.append(accuracy)
    precisions_array.append(classification_report_dict['macro avg']['precision'])
    recalls_array.append(classification_report_dict['macro avg']['recall'])

    # Dispaying the results of the metrics of the 5 folds
    print(f'Accuracy: {accuracy:.2f}')
    print(classification_report(y_testData, y_predictData))

# Aggregate and displays the average rsults of the metrics using all the k-folds.
avg_accuracy = sum(accuracies_array) / len(accuracies_array)
avg_precision = sum(precisions_array) / len(precisions_array)
avg_recall = sum(recalls_array) / len(recalls_array)

# Final display of the results
print("\nAverage Metrics Across the 5 K-Folds:")
print(f'Average Accuracy of the dataset is: {avg_accuracy:.2f}')
print(f'Average Precision of the dataset is: {avg_precision:.2f}')
print(f'Average Recall of the dataset is: {avg_recall:.2f}')


Accuracy: 0.72
              precision    recall  f1-score   support

           0       0.56      0.57      0.56        96
           1       0.62      0.64      0.63       117
           2       0.62      0.64      0.63       118
           3       0.66      0.64      0.65       118
           4       0.83      0.69      0.75       116
           5       0.82      0.82      0.82       118
           6       0.71      0.79      0.75       117
           7       0.77      0.71      0.74       119
           8       0.74      0.77      0.75       120
           9       0.51      0.87      0.65       120
          10       0.93      0.82      0.88       120
          11       0.93      0.74      0.82       119
          12       0.67      0.75      0.71       119
          13       0.79      0.81      0.80       119
          14       0.80      0.81      0.80       118
          15       0.70      0.78      0.74       119
          16       0.72      0.78      0.75       109
          17

### Decision Trees

##### Dataset 1 (20 Newsgroup):

In [7]:
#train and test data for newsgroups dataset
X_train, X_test, y_train, y_test = train_test_split(d1_tfidf, d1_labels, test_size=0.2, random_state=42)

decision_tree_classifier = DecisionTreeClassifier(criterion="gini", splitter="best", random_state=42, min_samples_split=10)

decision_tree_classifier.fit(X_train, y_train)

predicted_y = decision_tree_classifier.predict(X_test)

dt_accuracy = metrics.accuracy_score(y_test, predicted_y)
dt_precision = metrics.precision_score(y_test, predicted_y, average="weighted")
dt_recall = metrics.recall_score(y_test, predicted_y, average="weighted")
dt_f1_score = metrics.f1_score(y_test, predicted_y, average="weighted")

print(f"Accuracy of Decision Tree: {dt_accuracy:.4f}")
print(f"Precision of Decision Tree: {dt_precision:.4f}")
print(f"Recall of Decision Tree: {dt_recall:.4f}")
print(f"F1 Score of Decision Tree: {dt_f1_score:.4f}")

Accuracy of Decision Tree: 0.7832
Precision of Decision Tree: 0.7843
Recall of Decision Tree: 0.7832
F1 Score of Decision Tree: 0.7830


##### Dataset 2 (IMDB Reviews):

In [8]:
#train and test for imdb dataset
X_train, X_test, y_train, y_test = train_test_split(d2_tfidf, d2_labels, test_size=0.2, random_state=42)

decision_tree_classifier = DecisionTreeClassifier(criterion="gini", splitter="best", random_state=42, min_samples_leaf=10, max_leaf_nodes=110, min_samples_split=20)

decision_tree_classifier.fit(X_train, y_train)

predicted_y = decision_tree_classifier.predict(X_test)

dt_accuracy = metrics.accuracy_score(y_test, predicted_y)
dt_precision = metrics.precision_score(y_test, predicted_y, average="weighted")
dt_recall = metrics.recall_score(y_test, predicted_y, average="weighted")
dt_f1_score = metrics.f1_score(y_test, predicted_y, average="weighted")

print(f"Accuracy of Decision Tree: {dt_accuracy:.4f}")
print(f"Precision of Decision Tree: {dt_precision:.4f}")
print(f"Recall of Decision Tree: {dt_recall:.4f}")
print(f"F1 Score of Decision Tree: {dt_f1_score:.4f}")

Accuracy of Decision Tree: 0.7444
Precision of Decision Tree: 0.7448
Recall of Decision Tree: 0.7444
F1 Score of Decision Tree: 0.7443


### Support Vector Machines

##### Dataset 1 (20 Newsgroup):

In [14]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score,  KFold 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

svm_model = LinearSVC(dual=False)

kf = KFold(n_splits=5)

model_results = {
    "Dataset": [],
    "Accuracy Score": [],
    "Precision Score": [],
    "Recall Score": [],
    "F1 Score": []
}

svm_scores_20newsgroup = cross_val_score(svm_model, d1_tfidf, d1_labels, cv=kf)
Y_pred_20newsgroup = svm_model.fit(d1_tfidf, d1_labels).predict(d1_tfidf)

svm_accuracy = metrics.accuracy_score(Y_pred_20newsgroup, d1_labels)
svm_precision = (metrics.precision_score(Y_pred_20newsgroup,d1_labels, average='weighted'))
svm_recall = (recall_score(Y_pred_20newsgroup, d1_labels, average='weighted'))
svm_f1_score = (f1_score(Y_pred_20newsgroup, d1_labels, average='weighted'))

print(f"Accuracy: {svm_accuracy:.4f}")
print(f"Precision: {svm_precision:.4f}")
print(f"Recall: {svm_recall:.4f}")
print(f"F1 Score: {svm_f1_score:.4f}")

Accuracy: 0.9996
Precision: 0.9996
Recall: 0.9996
F1 Score: 0.9996


##### Dataset 2 (IMDB Reviews):

In [15]:
Y_pred_imdb = svm_model.fit(d2_tfidf, d2_labels).predict(d2_tfidf)

svm_accuracy = metrics.accuracy_score(d2_labels,Y_pred_imdb)
svm_precision = (metrics.precision_score(d2_labels,Y_pred_imdb, average='weighted'))
svm_recall = (recall_score(d2_labels, Y_pred_imdb,average='weighted'))
svm_f1_score = (f1_score(d2_labels, Y_pred_imdb, average='weighted'))

print(f"Accuracy: {svm_accuracy:.4f}")
print(f"Precision: {svm_precision:.4f}")
print(f"Recall: {svm_recall:.4f}")
print(f"F1 Score: {svm_f1_score:.4f}")

Accuracy: 0.9901
Precision: 0.9901
Recall: 0.9901
F1 Score: 0.9901


### Ada Boost

##### Dataset 1 (20 Newsgroup):

In [26]:
processed_train_data, processed_test_data, y_train, y_test = train_test_split(d1_tfidf, d1_labels, test_size=0.2, random_state=42)

for estimators in [3,10,50,100,300,500]:
    boost = AdaBoostClassifier(n_estimators = estimators, learning_rate = 0.5)
    model = boost.fit(processed_train_data, y_train)

    y_pred = model.predict(processed_test_data)

    print("Accuracy with number of estimators", estimators, "is:", metrics.accuracy_score(y_pred, y_test))

for alpha in [0.1,0.5,0.8,1,1.5,2,3]: #note that from 2 onwards produces the exact same accuracy
    boost = AdaBoostClassifier(n_estimators = 300, learning_rate = alpha)
    model = boost.fit(processed_train_data, y_train)

    y_pred = model.predict(processed_test_data)

    print("Accuracy with learning rate:", alpha, "is:", metrics.accuracy_score(y_pred, y_test))

boost = AdaBoostClassifier(n_estimators = 1000, learning_rate = 0.5)
model = boost.fit(processed_train_data, y_train)

y_pred = model.predict(processed_test_data)

print("Accuracy:", metrics.accuracy_score(y_pred, y_test))
print("Precision:", metrics.precision_score(y_pred, y_test,average='weighted'))
print("Recall:", metrics.recall_score(y_pred, y_test,average='weighted'))
print("F1 Score:", metrics.f1_score(y_pred, y_test,average='weighted'))

Testing IMDb Dataset
Accuracy with number of estimators 3 is: 0.5287610619469026
Accuracy with number of estimators 10 is: 0.7101769911504425
Accuracy with number of estimators 50 is: 0.754424778761062
Accuracy with number of estimators 100 is: 0.8097345132743363
Accuracy with number of estimators 300 is: 0.8230088495575221
Accuracy with number of estimators 500 is: 0.834070796460177
Accuracy with learning rate: 0.1 is: 0.8539823008849557
Accuracy with learning rate: 0.5 is: 0.8230088495575221
Accuracy with learning rate: 0.8 is: 0.7566371681415929
Accuracy with learning rate: 1 is: 0.7765486725663717
Accuracy with learning rate: 1.5 is: 0.8097345132743363
Accuracy with learning rate: 2 is: 0.6902654867256637
Accuracy with learning rate: 3 is: 0.4358407079646018
Accuracy: 0.8584070796460177
Precision: 0.8693052375994887
Recall: 0.8584070796460177
F1 Score: 0.8580173961731085


##### Dataset 2 (IMDB Reviews):

In [27]:
processed_train_data, processed_test_data, y_train, y_test = train_test_split(d2_tfidf, d2_labels, test_size=0.2, random_state=42)

for estimators in [3,10,50,100,300,500]:
    boost = AdaBoostClassifier(n_estimators = estimators, learning_rate = 0.5)
    model = boost.fit(processed_train_data, y_train)

    y_pred = model.predict(processed_test_data)

    print("Accuracy with number of estimators", estimators, "is:", metrics.accuracy_score(y_pred, y_test))

for alpha in [0.1,0.5,0.8,1,1.5,2,3]: #note that from 2 onwards produces the exact same accuracy
    boost = AdaBoostClassifier(n_estimators = 300, learning_rate = alpha)
    model = boost.fit(processed_train_data, y_train)

    y_pred = model.predict(processed_test_data)

    print("Accuracy with learning rate:", alpha, "is:", metrics.accuracy_score(y_pred, y_test))

boost = AdaBoostClassifier(n_estimators = 300, learning_rate = 0.5)
model = boost.fit(processed_train_data, y_train)

y_pred = model.predict(processed_test_data)

print("Accuracy:", metrics.accuracy_score(y_pred, y_test))
print("Precision:", metrics.precision_score(y_pred, y_test,average='weighted'))
print("Recall:", metrics.recall_score(y_pred, y_test,average='weighted'))
print("F1 Score:", metrics.f1_score(y_pred, y_test,average='weighted'))

Accuracy with number of estimators 3 is: 0.655
Accuracy with number of estimators 10 is: 0.7282
Accuracy with number of estimators 50 is: 0.8066
Accuracy with number of estimators 100 is: 0.8318
Accuracy with number of estimators 300 is: 0.8636
Accuracy with number of estimators 500 is: 0.8686
Accuracy with learning rate: 0.1 is: 0.8216
Accuracy with learning rate: 0.5 is: 0.8636
Accuracy with learning rate: 0.8 is: 0.8576
Accuracy with learning rate: 1 is: 0.8524


KeyboardInterrupt: 

### Random Forest

##### Dataset 1 (20 Newsgroup):

In [27]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(d1_tfidf, d1_labels, test_size=0.2, random_state=42)

# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier on the training data
rf_classifier.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the performance of the classifier
rf_accuracy = metrics.accuracy_score(y_test, y_pred)
rf_precision = metrics.precision_score(y_test, y_pred, average='weighted')
rf_recall = metrics.recall_score(y_test, y_pred, average='weighted')
rf_f1_score = metrics.f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {rf_accuracy:.4f}")
print(f"Precision: {rf_precision:.4f}")
print(f"Recall: {rf_recall:.4f}")
print(f"F1 Score: {rf_f1_score:.4f}")


Accuracy: 0.8606
Precision: 0.8803
Recall: 0.8606
F1 Score: 0.8618


##### Dataset 2 (IMDB Reviews):

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(d2_tfidf, d2_labels, test_size=0.2, random_state=42)

# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the classifier on the training data
rf_classifier.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the performance of the classifier
rf_accuracy = metrics.accuracy_score(y_test, y_pred)
rf_precision = metrics.precision_score(y_test, y_pred, average='weighted')
rf_recall = metrics.recall_score(y_test, y_pred, average='weighted')
rf_f1_score = metrics.f1_score(y_test, y_pred, average='weighted')

print(f"Accuracy: {rf_accuracy:.4f}")
print(f"Precision: {rf_precision:.4f}")
print(f"Recall: {rf_recall:.4f}")
print(f"F1 Score: {rf_f1_score:.4f}")


Accuracy: 0.8372
Precision: 0.8372
Recall: 0.8372
F1 Score: 0.8372


## Validation

In [36]:
from sklearn.model_selection import cross_val_score, KFold
import pandas as pd

# Define models with their hyperparameters
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'AdaBoost': AdaBoostClassifier(n_estimators=50, random_state=42),
    'SVM': SVC(kernel='linear', probability=True, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42)
}

# Initialize KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Create a DataFrame to store performance metrics
results_df = pd.DataFrame(columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Score'])

# Model validation pipeline
for model_name, model in models.items():
    accuracy_scores = cross_val_score(model, X_tfidf, Y, cv=kf, scoring='accuracy')
    precision_scores = cross_val_score(model, X_tfidf, Y, cv=kf, scoring='precision_weighted')
    recall_scores = cross_val_score(model, X_tfidf, Y, cv=kf, scoring='recall_weighted')
    f1_scores = cross_val_score(model, X_tfidf, Y, cv=kf, scoring='f1_weighted')

    results_df = results_df.append({
        'Model': model_name,
        'Accuracy': accuracy_scores.mean(),
        'Precision': precision_scores.mean(),
        'Recall': recall_scores.mean(),
        'F1 Score': f1_scores.mean()
    }, ignore_index=True)

# Print the results table
print(results_df)

# Identify the best model for each metric
best_accuracy_model = results_df.loc[results_df['Accuracy'].idxmax()]['Model']
best_precision_model = results_df.loc[results_df['Precision'].idxmax()]['Model']
best_recall_model = results_df.loc[results_df['Recall'].idxmax()]['Model']
best_f1_model = results_df.loc[results_df['F1 Score'].idxmax()]['Model']

print("\nBest Models:")
print(f"Accuracy: {best_accuracy_model}")
print(f"Precision: {best_precision_model}")
print(f"Recall: {best_recall_model}")
print(f"F1 Score: {best_f1_model}")


C:\Users\twbm\AppData\Local\Temp\ipykernel_11924\1104062206.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\twbm\AppData\Local\Temp\ipykernel_11924\1104062206.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\twbm\AppData\Local\Temp\ipykernel_11924\1104062206.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\twbm\AppData\Local\Temp\ipykernel_11924\1104062206.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


                 Model  Accuracy  Precision    Recall  F1 Score
0        Random Forest  0.880811   0.895755  0.880811  0.881001
1             AdaBoost  0.739916   0.755450  0.739916  0.740215
2                  SVM  0.963663   0.964693  0.963663  0.963687
3        Decision Tree  0.780245   0.781392  0.780245  0.780037
4  Logistic Regression  0.945496   0.948900  0.945496  0.945737

Best Models:
Accuracy: SVM
Precision: SVM
Recall: SVM
F1 Score: SVM


C:\Users\twbm\AppData\Local\Temp\ipykernel_11924\1104062206.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
